# KH-5 ARGON GCPs retrieval and storage
The objective of this script is to retrieve all the GCPs defined for each available ARGON image, and merge them into a single `.shp` file, in order to display them with QGIS.

In [ ]:
import os
import posixpath
import posixpath
import pandas as pd
import geopandas as gpd

List of available KH-5 ARGON images:

In [3]:
images_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/KH-5_ARGON_images'
products = []

for x in os.listdir(images_root):
    if os.path.isdir(posixpath.join(images_root, x)):
        products.append(x)
        
products

['DS09034A007MC018',
 'DS09034A007MC019',
 'DS09034A008MC019',
 'DS09034A008MC020',
 'DS09034A008MC021',
 'DS09034A008MC022',
 'DS09058A024MC012',
 'DS09058A024MC013']

Find and concatenate all the GCPs:

In [9]:
all_GCPs = []
for product in products:
    file = posixpath.join(images_root, product, "GCP.points")
    if os.path.exists(file):
        GCPs_p = pd.read_csv(file, encoding="windows-1252", skiprows=1)
        GCPs_p.columns = ["lon", "lat", "x_img", "y_img", "enable", "dX", "dY", "residual"]
        GCPs_p.drop(columns=["enable", "dX", "dY", "residual"], inplace=True)
        GCPs_p.loc[:, "y_img"] = - GCPs_p.loc[:, "y_img"]
        GCPs_p.loc[:, "image"] = product
        all_GCPs.append(GCPs_p)

GCPs = pd.concat(all_GCPs, ignore_index=True)
GCPs.describe(percentiles=[])

,lon,lat,x_img,y_img
count,212.000000,212.000000,212.000000,212.000000
mean,15.328407,78.582534,9334.686023,9891.061621
std,2.396071,0.568299,1340.888413,3399.139763
min,11.453661,76.443669,5890.536830,1384.907798
50%,15.128492,78.613415,9335.673659,9938.286351
max,25.494722,79.501223,11825.378197,16322.224339


Save as a `.shp` file:

In [10]:
GCPs_gpd = gpd.GeoDataFrame(
    GCPs,
    geometry=gpd.points_from_xy(GCPs.lon, GCPs.lat),
    crs="EPSG:4326"
)

GCPs_gpd.to_file(
    posixpath.join(images_root, "GCPs.shp"),
    driver="ESRI Shapefile"
)